# Riemannian Tensor Completion with T3F

This notebook replicates the results of the paper *Low-rank tensor approximation for Chebyshev interpolation in parametric option pricing* by Kathrin Glau, Daniel Kressner, and Francesco Statti. The goal is to generalize the Riemannian tensor completion algorithm to interpolate tensorized functions of higher dimensionality (around 10) and complexity, with applications to high-dimensional Credit Valuation Adjustment (CVA) modeling.

The core techniques involve the Tensor Train (TT) format, Riemannian Conjugate Gradient optimization, and Chebyshev interpolation.

# Installation

To properly install the `t3f` package, you must run the following two cells and then restart the session so that the updated files take effect.

In [ ]:
!pip install t3f

import os

# Ruta al archivo initializers.py
init_file_path = '/usr/local/lib/python3.11/dist-packages/t3f/initializers.py'

# Nueva versión de random_tensor con docstring correctamente formateado
new_random_tensor = '''def random_tensor(shape, tt_rank=2, mean=0., stddev=1., dtype=tf.float32,
                  name='t3f_random_tensor'):
    """Generate a random TT-tensor of the given shape with given mean and stddev.

    Entries of the generated tensor (in the full format) will be iid and satisfy
    E[x_{i1i2..id}] = mean, Var[x_{i1i2..id}] = stddev^2, but the distribution is
    in fact not Gaussian (but is close for large tensors).

    In the current implementation only mean 0 is supported. To get
    a random_tensor with specified mean but tt_rank greater by 1 you can
    call:
        x = t3f.random_tensor(shape, tt_rank, stddev=stddev)
        x = mean * t3f.ones_like(x) + x

    Args:
        shape: array representing the shape of the future tensor.
        tt_rank: a number or a (d+1)-element array with the desired ranks.
        mean: a number, the desired mean for the distribution of entries.
        stddev: a number, the desired standard deviation for the distribution of
            entries.
        dtype: [tf.float32] dtype of the resulting tensor.
        name: string, name of the Op.

    Returns:
        TensorTrain containing a TT-tensor
    """
    shape = np.array(shape)
    tt_rank = np.array(tt_rank)
    _validate_input_parameters(is_tensor=True, shape=shape, tt_rank=tt_rank)

    num_dims = shape.size
    if tt_rank.size == 1:
        tt_rank = tt_rank * np.ones(num_dims - 1, dtype=int)
        tt_rank = np.insert(tt_rank, 0, 1)
        tt_rank = np.append(tt_rank, 1)

    tt_rank = tt_rank.astype(int)

    # Empirically entries of a TT tensor with cores initialized from N(0, 1)
    # will have variances np.prod(tt_rank) and mean 0.
    # We scale each TT-core to obtain the desired stddev
    cr_exponent = -1.0 / (2 * num_dims)
    var = np.prod(tt_rank ** cr_exponent)
    core_stddev = stddev ** (1.0 / num_dims) * var
    with tf.name_scope(name):
        tt = tensor_with_random_cores(shape, tt_rank=tt_rank, stddev=core_stddev,
                                      dtype=dtype)

    if np.abs(mean) < 1e-8:
        return tt
    else:
        raise NotImplementedError('non-zero mean is not supported yet')
'''

# Verificar si el archivo existe y modificarlo
if os.path.exists(init_file_path):
    # Leer el contenido actual
    with open(init_file_path, 'r') as f:
        content = f.read()

    # 1. Añadir el nuevo import después de "from t3f import shapes"
    import_line = 'from t3f import shapes'
    new_import = 'from numbers import Integral as integer'
    if new_import not in content:
        # Asegurarse de que el import se añada solo una vez y en la posición correcta
        content = content.replace(import_line, f"{import_line}\n{new_import}", 1)

    # 2. Reemplazar la función random_tensor de manera más robusta
    start_marker = 'def random_tensor('
    try:
        start_idx = content.index(start_marker)
        # Buscar el final de la función buscando la siguiente definición o el final del archivo
        next_func_marker = 'def '
        end_idx = content.find(next_func_marker, start_idx + len(start_marker))
        if end_idx == -1:  # Si no hay más funciones, ir al final del archivo
            end_idx = len(content)
        else:
            # Retroceder hasta encontrar el final real de la función (antes de la próxima definición)
            end_idx = content.rfind('\n\n', start_idx, end_idx)
            if end_idx == -1:
                end_idx = content.rfind('\n', start_idx, end_idx)

        # Construir el nuevo contenido
        new_content = content[:start_idx] + new_random_tensor + content[end_idx:]

        # Escribir el contenido modificado
        with open(init_file_path, 'w') as f:
            f.write(new_content)

        print(f"El archivo {init_file_path} ha sido modificado exitosamente!")
    except ValueError as e:
        print(f"Error al encontrar marcadores en el archivo: {e}")
else:
    print(f"El archivo {init_file_path} no existe o no es accesible.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for t3f: filename=t3f-1.2.0-py3-none-any.whl size=69159 sha256=6b39e92c76a619625008eaf6921166aa05378abc7898824ac5e7ce9b52bfc5e8
  Stored in directory: /root/.cache/pip/wheels/36/ce/e5/723e43d13ef4de2e16b41e4f1aca48d999fc1528b3538277f5
Successfully built t3f
El archivo /usr/local/lib/python3.11/dist-packages/t3f/initializers.py ha sido modificado exitosamente!


This cell is used to check that the files have been correctly modified.

If you see the following line in the import section at the beginning of the file, it means everything is set up properly:

```
from numbers import Integral as integer
```

In [ ]:
import os
import t3f

# Ruta específica al archivo __init__.py
t3f_initializers_path = '/usr/local/lib/python3.11/dist-packages/t3f/initializers.py'

# Verificar si el archivo existe y mostrar su contenido
if os.path.exists(t3f_initializers_path):
    print(f"Contenido del archivo {t3f_initializers_path}:\n")
    with open(t3f_initializers_path, 'r') as f:
        content = f.read()
        print(content)
else:
    print(f"El archivo {t3f_initializers_path} no existe o no es accesible.")

Contenido del archivo /usr/local/lib/python3.11/dist-packages/t3f/initializers.py:

import numpy as np
import tensorflow as tf

from t3f.tensor_train import TensorTrain
from t3f.tensor_train_batch import TensorTrainBatch
from t3f.tensor_train_base import TensorTrainBase
from t3f import shapes
from numbers import Integral as integer


def _validate_input_parameters(is_tensor, shape, **params):
  """Internal function for validating input parameters

  Args:
    is_tensor: bool, determines whether we attempt to construct a TT-tensor or
      a TT-matrix (needed for the correct shape checks).
    shape: array, the desired shape of the generated TT object
    params: optional, possible values:
      batch_size: int, for constructing batches
      tt_rank: array or int, desired TT-ranks
  """

  if is_tensor:
    if len(shape.shape) != 1:
      raise ValueError('shape should be 1d array, got %a' % shape)
    if np.any(shape < 1):
      raise ValueError('all elements in `shape` should be posi

---

# Auxiliary functions (**IMPORTANT**)

In [ ]:
import tensorflow as tf
import t3f
import functools
import numpy as np

def objective_function(X, A, Omega):
    X_projected = t3f.gather_nd(X, Omega)  # Correct: use t3f.gather_nd for TensorTrain
    A_projected = tf.gather(A, tf.range(tf.shape(Omega)[0]))
    Z = X_projected - A_projected

    # Compute inner product of Z with itself
    inner_product_Z = tf.tensordot(Z, Z, axes=1)

    return 0.5 * inner_product_Z

# Automatic Riemannian gradient
def compute_riemannian_gradient_tf(X, A, Omega):
    X_ortho = t3f.orthogonalize_tt_cores(X, left_to_right=True)
    partial_obj_fn = functools.partial(objective_function, A=A, Omega=Omega)
    gradient = t3f.gradients(partial_obj_fn, X_ortho, runtime_check=False)
    return gradient  # Returns a TensorTrain object

def linearized_search(A_Omega, X_k, eta_k, Omega):
    X_k_proj = t3f.gather_nd(X_k, Omega)
    eta_k_proj = t3f.gather_nd(eta_k, Omega)

    A_Omega = tf.constant(A_Omega, dtype=tf.float32)
    diff = A_Omega - X_k_proj

    numerator = tf.tensordot(eta_k_proj, diff, axes=1)
    denominator = tf.tensordot(eta_k_proj, eta_k_proj, axes=1)

    alpha_k = numerator / denominator
    return alpha_k

def increase_tt_rank_mu(tt_tensor, mu, noise_magnitude=1e-8):
    """
    Increases the TT-rank at position mu+1 following algorithm (4.7).

    Args:
        tt_tensor (t3f.TensorTrain): Tensor in Tensor Train format.
        mu (int): Index where the rank is increased. Valid values: mu = 1,...,d-1
        noise_magnitude (float, optional): Magnitude of random vectors R_mu and R_{mu+1}. Default = 1e-8.

    Returns:
        t3f.TensorTrain: TT tensor with increased rank at position mu+1.
    """
    tt_ortho = t3f.orthogonalize_tt_cores(tt_tensor, left_to_right=True)
    tt_cores = tt_ortho.tt_cores
    U_L = tt_cores[mu - 1]
    U_R = tt_cores[mu]

    tt_ranks = t3f.tt_ranks(tt_tensor)
    r_mu_minus_1 = tt_ranks[mu - 1]
    r_mu = tt_ranks[mu]
    r_mu_plus_1 = tt_ranks[mu + 1]

    n_mu = tt_tensor.get_shape()[mu - 1]
    n_mu_plus_1 = tt_tensor.get_shape()[mu]

    R_mu = tf.random.normal([r_mu_minus_1 * n_mu, 1], mean=0.0, stddev=noise_magnitude)
    R_mu_plus_1 = tf.random.normal([1, r_mu_plus_1 * n_mu_plus_1], mean=0.0, stddev=noise_magnitude)

    R_mu = tf.reshape(R_mu, [r_mu_minus_1, n_mu, 1])
    R_mu_plus_1 = tf.reshape(R_mu_plus_1, [1, n_mu_plus_1, r_mu_plus_1])

    U_L_new = tf.concat([U_L, R_mu], axis=2)
    U_R_new = tf.concat([U_R, R_mu_plus_1], axis=0)

    new_cores = tt_cores[:mu - 1] + (U_L_new, U_R_new) + tt_cores[mu + 1:]

    return t3f.TensorTrain(new_cores)

def truncate(tensor, target_ranks):
    """Truncates a TT tensor to the target TT-ranks."""
    return t3f.round(tensor, max_tt_rank=target_ranks)

def riemannian_tensor_completion(X_0, A_Omega, Omega, max_iters=10, verbose=False):
    X_k = t3f.orthogonalize_tt_cores(X_0, left_to_right=True)
    target_ranks = t3f.tt_ranks(X_0).numpy()

    xi_0 = compute_riemannian_gradient_tf(X_k, A_Omega, Omega)
    eta_0 = -xi_0
    alpha_0 = linearized_search(A_Omega, X_k, eta_0, Omega)

    X_temp = X_k + alpha_0 * eta_0
    X_k = t3f.orthogonalize_tt_cores(truncate(X_temp, target_ranks), left_to_right=True)

    xi_prev = xi_0
    eta_prev = eta_0
    ip_xi_xi_old = t3f.frobenius_norm_squared(xi_0)

    for k in range(1, max_iters + 1):
        xi_k = compute_riemannian_gradient_tf(X_k, A_Omega, Omega)
        ip_xi_xi = t3f.frobenius_norm_squared(xi_k)

        eta_transported = t3f.project(eta_prev, X_k)
        beta_k = ip_xi_xi / ip_xi_xi_old if ip_xi_xi_old != 0 else 0

        eta_transported = truncate(eta_transported, target_ranks)
        eta_k = -xi_k + beta_k * eta_transported
        eta_k = truncate(eta_k, target_ranks)

        alpha_k = linearized_search(A_Omega, X_k, eta_k, Omega)
        X_temp = X_k + alpha_k * eta_k
        X_k = t3f.orthogonalize_tt_cores(truncate(X_temp, target_ranks), left_to_right=True)

        xi_prev = xi_k
        eta_prev = eta_k
        ip_xi_xi_old = ip_xi_xi

    return X_k

def chebyshev_nodes(n):
    """
    Generate Chebyshev nodes of the first kind in the interval [0, 1].

    Parameters:
    - n (int): Number of intervals (number of nodes will be n + 1).

    Returns:
    - np.array: Array of (n + 1) Chebyshev nodes in [0, 1].
    """
    k = np.arange(n + 1)
    q = np.cos(np.pi * k / n)
    x = (q + 1) / 2
    return x

def make_omega_set(n_nodes, size, d):
    """
    Generate a set of random multi-indices with d components, each between 0 and n_nodes - 1.

    Parameters:
    - n_nodes (int): Number of discrete points per dimension.
    - size (int): Number of multi-indices to generate.
    - d (int): Number of dimensions.

    Returns:
    - np.array: 2D array of shape (size, d) with random multi-indices.
    """
    if n_nodes < 1:
        raise ValueError("n_nodes must be positive")
    if size < 0:
        raise ValueError("size must be non-negative")

    total_combinations = n_nodes ** d
    if size > total_combinations:
        raise ValueError(f"Requested size ({size}) exceeds total possible combinations ({total_combinations})")

    Omega = np.random.randint(0, n_nodes, size=(size, d))
    return Omega

def map_to_chebyshev_nodes(Omega, n_nodes):
    """
    Map multi-indices from range [0, n_nodes - 1] to Chebyshev nodes in [0, 1] for each dimension.

    Parameters:
    - Omega: 2D NumPy array of shape (size, d) with multi-indices.
    - n_nodes: Number of Chebyshev nodes per dimension.

    Returns:
    - 2D NumPy array of shape (size, d) with mapped Chebyshev nodes.
    """
    if n_nodes < 1:
        raise ValueError("n_nodes must be positive")

    cheb_nodes = chebyshev_nodes(n_nodes - 1)
    d = Omega.shape[1]
    Omega_mapped = np.zeros_like(Omega, dtype=float)
    for dim in range(d):
        Omega_mapped[:, dim] = cheb_nodes[Omega[:, dim]]

    return Omega_mapped

def infer_input_dimension(f_vectorized, max_dimension=100):
    """
    Try to infer the input dimension of a vectorized function f.

    Parameters:
    - f_vectorized (callable): A function that takes input of shape (n_samples, d).
    - max_dimension (int): Maximum dimension to try.

    Returns:
    - int or None: Inferred input dimension, or None if not found.
    """
    for d in range(1, max_dimension + 1):
        try:
            test_input = np.zeros((1, d))
            f_vectorized(test_input)
            return d
        except (ValueError, IndexError):
            continue
        except Exception:
            return None
    return None

def generate_disjoint_omega_c(Omega, sizeOmega_C, number_nodes):
    """
    Generate a disjoint set Omega_C of the specified size.

    Parameters:
    - Omega (array): Existing indices.
    - sizeOmega_C (int): Number of disjoint indices to generate.
    - number_nodes (int): Number of discrete values per dimension.

    Returns:
    - np.array: Array of shape (sizeOmega_C, d) with disjoint indices.
    """
    sizeOmega = len(Omega)
    d = Omega.shape[1]

    if sizeOmega_C > number_nodes ** d - sizeOmega:
        raise ValueError("sizeOmega_C is too large to generate disjoint multi-indices")

    Omega = Omega.numpy() if isinstance(Omega, tf.Tensor) else Omega
    Omega_set = set(map(tuple, Omega))

    Omega_C = []
    candidates_per_batch = int(sizeOmega_C * 1.5)
    attempts = 0
    max_attempts = 10

    while len(Omega_C) < sizeOmega_C and attempts < max_attempts:
        candidates = np.random.randint(0, number_nodes, size=(candidates_per_batch, d))
        for idx in candidates:
            idx_tuple = tuple(idx)
            if idx_tuple not in Omega_set and idx_tuple not in {tuple(x) for x in Omega_C}:
                Omega_C.append(idx)
                if len(Omega_C) >= sizeOmega_C:
                    break
        attempts += 1

    if len(Omega_C) < sizeOmega_C:
        raise ValueError(f"Could not generate {sizeOmega_C} disjoint indices after {max_attempts} attempts")

    return np.array(Omega_C)

In the next cell, we modify `adaptive_rank_strategy_Glau_fast` so that, in addition to the test set error, it also computes the training set error at each iteration (to check whether increasing the TT-rank helps reduce it).

Note that if it doesn’t even fit the training set properly, the model likely needs more complexity (higher TT-rank), and the acceptance criterion may be too strict.

I will also include the logic of `allow_error_augmentation` here.

In [ ]:
def adaptive_rank_strategy_Glau_fast(Omega, A_Omega, Omega_C, A_Omega_C, shape,
                                     max_rank, rank_initial, verbose,
                                     initial_guess=False, X_initial=None,
                                     max_iters=10, rho=1e-4,
                                     allow_error_augmentation=False):
    """
    Fast adaptive TT-rank strategy based on Steinlechner's Riemannian optimization paper.
    Evaluates training (Omega) and validation (Omega_C) errors at each iteration.
    Supports two rank increase acceptance criteria based on allow_error_augmentation.

    Parameters:
        Omega (np.ndarray): Indices of observed entries for training.
        A_Omega (np.ndarray): Values of observed training entries.
        Omega_C (np.ndarray): Validation/test indices.
        A_Omega_C (np.ndarray): Validation/test values.
        shape (tuple): Shape of the full tensor.
        max_rank (int): Maximum allowed TT-rank.
        rank_initial (int): Initial TT-rank used for random initialization.
        verbose (bool): If True, prints training progress.
        initial_guess (bool): Whether to use a custom initial tensor.
        X_initial (t3f.TensorTrain): Initial TT tensor (if initial_guess=True).
        max_iters (int): Max iterations for Riemannian CG.
        rho (float): Threshold for accepting rank increase.
        allow_error_augmentation (bool): Whether to allow error to increase slightly.

    Returns:
        tuple: (X, errors) where
            - X is the completed TT tensor,
            - errors is a list of validation set errors at each accepted rank update.
    """
    Omega = tf.constant(Omega, dtype=tf.int32)
    Omega_C = tf.constant(Omega_C, dtype=tf.int32)
    A_Omega = tf.constant(A_Omega, dtype=tf.float32)
    A_Omega_C = tf.constant(A_Omega_C, dtype=tf.float32)

    d = len(shape)

    if initial_guess:
        if X_initial is None:
            raise ValueError("X_initial must be provided when initial_guess=True.")
        X = X_initial
    else:
        initial_tt_rank = [1] + [rank_initial] * (d - 1) + [1]
        X = t3f.random_tensor(shape, tt_rank=initial_tt_rank)

    X = riemannian_tensor_completion(X, A_Omega, Omega, max_iters=max_iters, verbose=verbose)

    X_Omega_C = t3f.gather_nd(X, Omega_C)
    norm_A_C = np.linalg.norm(A_Omega_C.numpy())
    error_val = np.linalg.norm(A_Omega_C.numpy() - X_Omega_C.numpy()) / norm_A_C

    X_Omega = t3f.gather_nd(X, Omega)
    norm_A = np.linalg.norm(A_Omega.numpy())
    error_train = np.linalg.norm(A_Omega.numpy() - X_Omega.numpy()) / norm_A

    if verbose:
        print(f"Initial validation error: {error_val:.6f}")
        print(f"Initial training error: {error_train:.6f}")

    errors = [error_val]
    locked = 0
    mu = 1

    while locked < d - 1 and max(t3f.tt_ranks(X).numpy()[1:d]) < max_rank:
        ranks = t3f.tt_ranks(X).numpy()

        if ranks[mu] < max_rank:
            X_new = increase_tt_rank_mu(X, mu)
            X_new = riemannian_tensor_completion(X_new, A_Omega, Omega, max_iters=max_iters, verbose=verbose)

            X_new_Omega_C = t3f.gather_nd(X_new, Omega_C)
            error_val_new = np.linalg.norm(A_Omega_C.numpy() - X_new_Omega_C.numpy()) / norm_A_C

            X_new_Omega = t3f.gather_nd(X_new, Omega)
            error_train_new = np.linalg.norm(A_Omega.numpy() - X_new_Omega.numpy()) / norm_A

            if verbose:
                print(f"mu={mu}: New validation error = {error_val_new:.6f}")
                print(f"mu={mu}: New training error = {error_train_new:.6f}")

            if allow_error_augmentation:
                accept = error_train_new <= error_train + rho
            else:
                accept = error_train_new < error_train - rho

            if accept:
                X = X_new
                error_val = error_val_new
                error_train = error_train_new
                errors.append(error_val)
                locked = 0
                if verbose:
                    print(f"Rank increase accepted at mu={mu}.")
            else:
                locked += 1
                if verbose:
                    print(f"Rank increase rejected at mu={mu}.")
        else:
            locked += 1
            if verbose:
                print(f"Rank at mu={mu} is already at max.")

        mu = (mu % (d - 1)) + 1

    if verbose:
        print(f"Final validation error: {error_val:.6f}")
        print(f"Final training error: {error_train:.6f}")

    return X, errors

---

# Complete tensor completion algorithm

This cell implements Algorithm 1 from the adaptive sampling paper.

In [ ]:
def adaptive_sampling_strategy_1_v2(f_vectorized, sizeOmega, sizeOmega_C, number_nodes,
                                    rank_initial, max_rank, p=0.2,
                                    tol_1=1e-4, tol_2=1e-8,
                                    max_iters=10, rho=1e-4,
                                    verbose=False, allow_error_augmentation=False):
    """
    Adaptive sampling strategy for tensor completion.
    At each stage, refines the solution using additional samples and updates the training set.

    Parameters:
        f_vectorized (callable): Function to interpolate (vectorized).
        sizeOmega (int): Initial training set size.
        sizeOmega_C (int): Validation set size.
        number_nodes (int): Number of nodes per dimension.
        rank_initial (int): Initial TT-rank.
        max_rank (int): Maximum TT-rank allowed.
        p (float): Max percentage of observed entries vs full tensor size.
        tol_1 (float): Tolerance on absolute error (stopping criterion).
        tol_2 (float): Tolerance on error change between iterations.
        max_iters (int): Max Riemannian CG iterations.
        rho (float): Rank increase acceptance threshold.
        verbose (bool): If True, print logs.
        allow_error_augmentation (bool): If True, allows slight error increase when updating rank.

    Returns:
        t3f.TensorTrain: The final approximated TT tensor.
    """
    d = infer_input_dimension(f_vectorized)
    n = number_nodes * np.ones(d, dtype=int)
    shape = tuple(n)

    Omega = make_omega_set(number_nodes, sizeOmega, d)
    Omega_mapped = map_to_chebyshev_nodes(Omega, number_nodes)
    Omega_C = generate_disjoint_omega_c(Omega, sizeOmega_C, number_nodes)
    Omega_C_mapped = map_to_chebyshev_nodes(Omega_C, number_nodes)

    A_Omega = f_vectorized(Omega_mapped)
    A_Omega_C = f_vectorized(Omega_C_mapped)

    Omega = tf.constant(Omega, dtype=tf.int32)
    Omega_C = tf.constant(Omega_C, dtype=tf.int32)
    A_Omega = tf.constant(A_Omega, dtype=tf.float32)
    A_Omega_C = tf.constant(A_Omega_C, dtype=tf.float32)

    X_c, _ = adaptive_rank_strategy_Glau_fast(Omega, A_Omega, Omega_C, A_Omega_C,
                                              shape, max_rank, rank_initial,
                                              verbose=verbose, initial_guess=False,
                                              max_iters=max_iters, rho=rho,
                                              allow_error_augmentation=allow_error_augmentation)

    X_C_eval = t3f.gather_nd(X_c, Omega_C)
    err_new = np.linalg.norm(A_Omega_C.numpy() - X_C_eval.numpy()) / np.linalg.norm(A_Omega_C.numpy())
    if verbose:
        print(f"Iteration 0: |Omega| = {len(Omega)}, Validation error = {err_new:.6f}")

    iteration = 0
    while len(Omega) / np.prod(shape) < p:
        iteration += 1
        err_old = err_new

        Omega_C_old = Omega_C
        A_Omega_C_old = A_Omega_C

        Omega_C = generate_disjoint_omega_c(Omega_C_old, sizeOmega_C, number_nodes)
        Omega_C_mapped = map_to_chebyshev_nodes(Omega_C, number_nodes)
        A_Omega_C = f_vectorized(Omega_C_mapped)

        Omega_C = tf.constant(Omega_C, dtype=tf.int32)
        A_Omega_C = tf.constant(A_Omega_C, dtype=tf.float32)

        Omega = tf.concat([Omega, Omega_C_old], axis=0)
        A_Omega = tf.concat([A_Omega, A_Omega_C_old], axis=0)

        print(f"Iteration {iteration}: |Omega| = {tf.shape(Omega)[0].numpy()}")

        X_c, _ = adaptive_rank_strategy_Glau_fast(Omega, A_Omega, Omega_C, A_Omega_C,
                                                  shape, max_rank, rank_initial,
                                                  verbose=verbose, initial_guess=True,
                                                  X_initial=X_c, max_iters=max_iters,
                                                  rho=rho,
                                                  allow_error_augmentation=allow_error_augmentation)

        X_C_eval = t3f.gather_nd(X_c, Omega_C)
        err_new = np.linalg.norm(A_Omega_C.numpy() - X_C_eval.numpy()) / np.linalg.norm(A_Omega_C.numpy())

        tt_ranks = t3f.tt_ranks(X_c)
        rank_max_reached = any(r >= max_rank for r in tt_ranks[1:-1])

        if verbose:
            print(f"Iteration {iteration}: |Omega| = {len(Omega)}, Validation error = {err_new:.6f}, TT-ranks = {tt_ranks.numpy()}")

        if (err_new < tol_1 or abs(err_new - err_old) < tol_2 or rank_max_reached):
            break

    return X_c

---

# Test 1 (simple function from the paper)

The next cell defines the function to be interpolated. This specific function is used in Glau’s paper as a first experiment in tensor completion, but for our purposes it is too simple.

In [ ]:
np.random.seed(610014)
tf.random.set_seed(610014)

In [ ]:
def f_vectorized(X):
    """
    Función f(x) = exp(-||x||), donde x es un vector en [0,1]^4.
    X: array de forma (N, 4), donde N es el número de puntos.
    Retorna: array de forma (N,) con los valores de la función.
    """
    if X.shape[1] != 4:
        raise ValueError("La función espera entradas de dimensión 4")
    norm_x = np.linalg.norm(X, axis=1)  # Norma euclidiana a lo largo de la dimensión de los puntos
    return np.exp(-norm_x)

The following cell sets the parameters for running the algorithm (for this specific function) and executes the code.

In [ ]:
# Probar el código
if __name__ == "__main__":

    seed = 610015
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # Parámetros del algoritmo
    sizeOmega = 1600
    sizeOmega_C = 2000
    number_nodes = 20
    rank_initial = 1
    max_rank = 7
    p = 0.25
    tol_1 = 1e-3
    tol_2 = 1e-8
    max_iters = 10
    rho = 1e-4
    verbose = True

    # Ejecutar el algoritmo
    X = adaptive_sampling_strategy_1(
        f_vectorized=f_vectorized,
        sizeOmega=sizeOmega,
        sizeOmega_C=sizeOmega_C,
        number_nodes=number_nodes,
        rank_initial=rank_initial,
        max_rank=max_rank,
        p=p,
        tol_1=tol_1,
        tol_2=tol_2,
        max_iters=max_iters,
        rho=rho,
        verbose=verbose
    )

    print("Tensor completado X:", X)
    print("Rangos TT de X:", t3f.tt_ranks(X))

Error inicial en Omega_C: 1.057932
Aumento en mu=1: Rango TT de X_new: [1 2 1 1 1]
Aumento en mu=1: error_X_new = 1.376843
Aumento de rango rechazado para mu=1. Locked: 1
Aumento en mu=2: Rango TT de X_new: [1 1 2 1 1]
Aumento en mu=2: error_X_new = 1.760152
Aumento de rango rechazado para mu=2. Locked: 2
Aumento en mu=3: Rango TT de X_new: [1 1 1 2 1]
Aumento en mu=3: error_X_new = 1.270101
Aumento de rango rechazado para mu=3. Locked: 3
Error final en Omega_C: 1.057932
Iteración 0: |Omega| = 1600, Error en Omega_C = 1.057932
Iteración 1: Tamaño de Omega = 3600
Error inicial en Omega_C: 1.048903
Aumento en mu=1: Rango TT de X_new: [1 2 1 1 1]
Aumento en mu=1: error_X_new = 1.417135
Aumento de rango rechazado para mu=1. Locked: 1
Aumento en mu=2: Rango TT de X_new: [1 1 2 1 1]
Aumento en mu=2: error_X_new = 1.382957
Aumento de rango rechazado para mu=2. Locked: 2
Aumento en mu=3: Rango TT de X_new: [1 1 1 2 1]
Aumento en mu=3: error_X_new = 1.422563
Aumento de rango rechazado para mu=3

We now compare some elements of the original tensor with those of the completed tensor (the approximation). Note that the code below materializes the full original tensor, which is not feasible in very high dimensions.

For that, use `tf.gather_nd` or a similar method.

In [ ]:
import numpy as np
import tensorflow as tf
import t3f

# Assuming f_vectorized and adaptive_sampling_strategy_1 are defined as in your code

def generate_full_chebyshev_tensor(f_vectorized, number_nodes):
    """Generate the full tensor by evaluating f_vectorized on a Chebyshev grid."""
    d = inferir_dimension_entrada(f_vectorized)
    cheb_nodes = chebyshev_nodes(number_nodes - 1)  # n_nodes - 1 intervals give n_nodes points
    grid = np.meshgrid(*[cheb_nodes] * d, indexing='ij')
    grid_flat = np.stack([g.flatten() for g in grid], axis=-1)  # Shape: (number_nodes^d, d)
    values = f_vectorized(grid_flat)  # Shape: (number_nodes^d,)
    return values.reshape([number_nodes] * d)  # Shape: (n, n, ..., n)

def compare_random_samples(original_tensor, approx_tensor, N_samples):
    """Compare N_samples random elements from original and approximated tensors."""
    shape = original_tensor.shape
    d = len(shape)
    total_elements = np.prod(shape)

    # Generate N_samples random multi-indices
    indices = np.random.randint(0, shape[0], size=(N_samples, d))
    indices_tf = tf.constant(indices, dtype=tf.int32)

    # Extract values from original tensor (NumPy array)
    original_values = original_tensor[tuple(indices.T)]  # Shape: (N_samples,)

    # Extract values from approximated TT tensor using t3f.gather_nd
    approx_values = t3f.gather_nd(approx_tensor, indices_tf).numpy()  # Shape: (N_samples,)

    # Compute absolute errors
    abs_errors = np.abs(original_values - approx_values)

    # Display comparison
    print(f"\nComparing {N_samples} random samples:")
    print(f"{'Index':<20} {'Original':<15} {'Approximated':<15} {'Abs Error':<15}")
    print("-" * 65)
    for i in range(N_samples):
        idx_tuple = tuple(int(idx) for idx in indices[i])  # Convert indices to standard integers
        idx_str = str(idx_tuple)
        print(f"{idx_str:<20} {original_values[i]:<15.6f} {approx_values[i]:<15.6f} {abs_errors[i]:<15.6f}")

    # Summary statistics
    mean_abs_error = np.mean(abs_errors)
    max_abs_error = np.max(abs_errors)
    print(f"\nMean Absolute Error: {mean_abs_error:.6f}")
    print(f"Max Absolute Error: {max_abs_error:.6f}")

    return abs_errors

N_samples = 10

# Generate original tensor
original_tensor = generate_full_chebyshev_tensor(f_vectorized, number_nodes)
print(f"Original tensor shape: {original_tensor.shape}")

# Run adaptive sampling strategy
X_approx = X

print(f"Approximated tensor TT ranks: {t3f.tt_ranks(X_approx).numpy()}")

# Compare random samples
errors = compare_random_samples(original_tensor, X_approx, N_samples)

Original tensor shape: (20, 20, 20, 20)
Approximated tensor TT ranks: [1 5 4 4 1]

Comparing 10 random samples:
Index                Original        Approximated    Abs Error      
-----------------------------------------------------------------
(4, 8, 8, 10)        0.262758        0.262748        0.000010       
(2, 15, 2, 0)        0.181922        0.182091        0.000169       
(14, 9, 9, 4)        0.304710        0.304680        0.000030       
(6, 1, 4, 1)         0.159435        0.159287        0.000148       
(7, 10, 18, 11)      0.399046        0.399309        0.000263       
(7, 16, 3, 17)       0.309078        0.309050        0.000028       
(2, 7, 5, 0)         0.169941        0.169673        0.000268       
(3, 5, 2, 4)         0.161103        0.160962        0.000141       
(17, 14, 15, 9)      0.562580        0.562705        0.000125       
(14, 7, 8, 9)        0.334692        0.334552        0.000140       

Mean Absolute Error: 0.000132
Max Absolute Error: 0.000268


**IMPORTANT:** The stopping criterion that matters most for our purposes is the first one, so we should check whether it is properly defined. In principle, there’s no issue since it is defined exactly as in the paper, but we need to ensure that `tol_1` has a value that makes sense for our goals.

---

# Test 2 ($f_1$)

This is the $f_1$ function used in the Overleaf tests.

In [ ]:
def f_vectorized(x):
    """
    Args:
        x (numpy.ndarray): Vector de 10 dimensiones.

    Returns:
        float: Evaluación de la función.
    """
    if len(x.shape) == 1:
        x = x.reshape(1, -1)
    num_samples, d = x.shape
    if d != 10:
        raise ValueError("Se requieren exactamente 10 dimensiones")

    return (x[:, 0] ** 2 + np.exp(x[:, 1]) + np.log(x[:, 2] + 1) +
            1 / (x[:, 3] + 1) + x[:, 4] ** 3 + np.sqrt(x[:, 5]) +
            x[:, 6] * x[:, 7] + x[:, 8] ** 4 - np.exp(-x[:, 9]))

In [ ]:
# Probar el código
if __name__ == "__main__":

    seed = 610015
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # Parámetros del algoritmo
    sizeOmega = 5000
    sizeOmega_C = 1000
    number_nodes = 50
    rank_initial = 10
    max_rank = 20
    p = 0.25
    tol_1 = 1e-3
    tol_2 = 1e-8
    max_iters = 10
    rho = 1e-4
    verbose = True
    allow_error_augmentation = False

    # Ejecutar el algoritmo
    X = adaptive_sampling_strategy_1_v2(
        f_vectorized=f_vectorized,
        sizeOmega=sizeOmega,
        sizeOmega_C=sizeOmega_C,
        number_nodes=number_nodes,
        rank_initial=rank_initial,
        max_rank=max_rank,
        p=p,
        tol_1=tol_1,
        tol_2=tol_2,
        max_iters=max_iters,
        rho=rho,
        verbose=verbose,
        allow_error_augmentation=allow_error_augmentation
    )

    print("Tensor completado X:", X)
    print("Rangos TT de X:", t3f.tt_ranks(X))

Error inicial en Omega_C (prueba): 1.535708
Error inicial en Omega (entrenamiento): 0.195603
Aumento en mu=1: Rango TT de X_new: [ 1 11 10 10 10 10 10 10 10 10  1]
Aumento en mu=1: Error en Omega_C (prueba) = 1.531042
Aumento en mu=1: Error en Omega (entrenamiento) = 0.172301
Aumento de rango aceptado para mu=1.
Aumento en mu=2: Rango TT de X_new: [ 1 11 11 10 10 10 10 10 10 10  1]
Aumento en mu=2: Error en Omega_C (prueba) = 1.504150
Aumento en mu=2: Error en Omega (entrenamiento) = 0.134824
Aumento de rango aceptado para mu=2.
Aumento en mu=3: Rango TT de X_new: [ 1 11 11 11 10 10 10 10 10 10  1]
Aumento en mu=3: Error en Omega_C (prueba) = 1.554512
Aumento en mu=3: Error en Omega (entrenamiento) = 0.062608
Aumento de rango aceptado para mu=3.
Aumento en mu=4: Rango TT de X_new: [ 1 11 11 11 11 10 10 10 10 10  1]
Aumento en mu=4: Error en Omega_C (prueba) = 1.549019
Aumento en mu=4: Error en Omega (entrenamiento) = 0.044919
Aumento de rango aceptado para mu=4.
Aumento en mu=5: Rango 

**Current behavior analysis:** In the first iteration of `adaptive_sampling_strategy_1`, the training set error does not decrease because the approximation tensor lacks sufficient complexity (TT-rank), since the rank increase acceptance criterion is too strict.

As a result, all iterations of `adaptive_rank_strategy_Glau_fast` finish without improving the error on $\Omega$ (training set), and the algorithm proceeds to increase the training set size. However, this approach is flawed, as the algorithm should first fit the small training set properly before expanding it.

**Conclusion:** The algorithm needs a less strict stopping criterion.

Add an explicit counter for how many times the function $f_1$ is evaluated.

---

# Test 3 ($f_2$)

In [ ]:
def f_vectorized(x):
    """
    Args:
        x (numpy.ndarray): Vector de 10 dimensiones.

    Returns:
        float: Evaluación de la función f2.
    """
    if len(x.shape) == 1:
        x = x.reshape(1, -1)
    num_samples, d = x.shape
    if d != 10:
        raise ValueError("Se requieren exactamente 10 dimensiones")

    return (x[:, 0]**2 * x[:, 6]**2 * x[:, 9]**2 * np.exp(-x[:, 1]**4 - x[:, 5] - x[:, 7] * x[:, 8]) +
            np.log(1 + x[:, 3] + x[:, 2] * x[:, 7]) +
            (x[:, 0]**2 + x[:, 1] + x[:, 4]) / (1 + x[:, 5]**3 + x[:, 2]**3 + x[:, 8]**4))

# Probar el código
if __name__ == "__main__":

    seed = 610015
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # Parámetros del algoritmo
    sizeOmega = 2000
    sizeOmega_C = 1000
    number_nodes = 50
    rank_initial = 12
    max_rank = 20
    p = 0.25
    tol_1 = 1e-3
    tol_2 = 1e-8
    max_iters = 10
    rho = 1e-4
    verbose = True

    # Ejecutar el algoritmo
    X = adaptive_sampling_strategy_1(
        f_vectorized=f_vectorized,
        sizeOmega=sizeOmega,
        sizeOmega_C=sizeOmega_C,
        number_nodes=number_nodes,
        rank_initial=rank_initial,
        max_rank=max_rank,
        p=p,
        tol_1=tol_1,
        tol_2=tol_2,
        max_iters=max_iters,
        rho=rho,
        verbose=verbose
    )

    print("Tensor completado X:", X)
    print("Rangos TT de X:", t3f.tt_ranks(X))

---

# All code in a single cell

In [ ]:
import tensorflow as tf
import t3f
import functools
import numpy as np

def funcion_objetivo(X, A, Omega):
    X_proyectado = t3f.gather_nd(X, Omega)  # Correcto: usar t3f.gather_nd para TensorTrain
    A_proyectado = tf.gather(A, tf.range(tf.shape(Omega)[0]))
    Z = X_proyectado - A_proyectado

    # Calcular el producto escalar de Z consigo mismo
    producto_escalar_Z = tf.tensordot(Z, Z, axes=1)

    return 0.5 * producto_escalar_Z

# Gradiente Riemanniano automático
def calcular_gradiente_riemanniano_tf(X, A, Omega):
    X_ortho = t3f.orthogonalize_tt_cores(X, left_to_right=True)
    funcion_objetivo_parcial = functools.partial(funcion_objetivo, A=A, Omega=Omega)
    gradiente = t3f.gradients(funcion_objetivo_parcial, X_ortho, runtime_check=False)
    return gradiente # El resultado es un TensorTrain de t3f

def linearized_search(A_Omega, X_k, eta_k, Omega):
    X_k_proyectado = t3f.gather_nd(X_k, Omega)
    eta_k_proyectado = t3f.gather_nd(eta_k, Omega)

    A_Omega = tf.constant(A_Omega, dtype=tf.float32)
    resta = A_Omega - X_k_proyectado

    # Calcular el numerador y el denominador
    num = tf.tensordot(eta_k_proyectado, resta, axes=1)
    den = tf.tensordot(eta_k_proyectado, eta_k_proyectado, axes=1)

    alpha_k = num / den
    return alpha_k

def increase_tt_rank_mu(tt_tensor, mu, noise_magnitude=1e-8):
    """
    Incrementa el rango TT en la posición mu+1 siguiendo el algoritmo (4.7).

    Args:
        tt_tensor (t3f.TensorTrain): Tensor en formato TT.
        mu (int): Índice donde se incrementará el rango. Únicos valores posibles: mu = 1,...,d-1
        noise_magnitude(float, opcional): Magnitud de los vectores aleatorios R_mu y R_{mu+1}. Default = 1e-8.

    Returns:
        t3f.TensorTrain: Tensor TT con rango aumentado en la posición mu+1.
    """
    # Paso 1: Ortogonalizar desde la izquierda hasta la posición mu
    tt_ortho = t3f.orthogonalize_tt_cores(tt_tensor, left_to_right=True)

    # Paso 2: Extraer núcleos TT
    tt_cores = tt_ortho.tt_cores  # Lista de núcleos TT. Esto es un tf.Tensor, por eso podemos usar .tt_cores
    U_L = tt_cores[mu-1]            # Núcleo en la posición mu
    U_R = tt_cores[mu]          # Núcleo en la posición mu+1

    # Paso 3: Extraer el vector de rangos TT correctamente
    tt_ranks = t3f.tt_ranks(tt_tensor)
    r_mu_minus_1 = tt_ranks[mu-1]  # Rango TT en la posición mu-1
    r_mu = tt_ranks[mu]            # Rango TT en la posición mu
    r_mu_plus_1 = tt_ranks[mu+1]   # Rango TT en la posición mu+1

    # Extraer las dimensiones de los modos
    n_mu = tt_tensor.get_shape()[mu-1]       # Dimensión en la posición mu
    n_mu_plus_1 = tt_tensor.get_shape()[mu]  # Dimensión en la posición mu+1

    # Paso 4: Crear vectores aleatorios con la norma deseada
    R_mu = tf.random.normal(shape=[r_mu_minus_1 * n_mu, 1], mean=0.0, stddev=noise_magnitude)
    R_mu_plus_1 = tf.random.normal(shape=[1, r_mu_plus_1 * n_mu_plus_1], mean=0.0, stddev=noise_magnitude)

    # Paso 5: Redimensionar los vectores para que sean compatibles con la estructura TT
    R_mu = tf.reshape(R_mu, [r_mu_minus_1, n_mu, 1])  # Convertir a (r_{mu-1}, n_mu, 1)
    R_mu_plus_1 = tf.reshape(R_mu_plus_1, [1, n_mu_plus_1, r_mu_plus_1])  # Convertir a (1, n_{mu+1}, r_{mu+1})

    # Paso 6: Modificar los núcleos TT en las posiciones mu y mu+1
    U_L_new = tf.concat([U_L, R_mu], axis=2)  # Expandir en la tercera dimensión (r_mu+1)
    U_R_new = tf.concat([U_R, R_mu_plus_1], axis=0)  # Expandir en la primera dimensión (r_mu+1)

    # Paso 7: Reconstruir la lista de núcleos con las modificaciones
    new_cores = tt_cores[:mu-1] + (U_L_new, U_R_new) + tt_cores[mu+1:]

    # Mostrar la lista de cores y sus dimensiones (Opcional)
    # print("Lista de new_cores y sus dimensiones:")
    # for i, core in enumerate(new_cores):
        # print(f"Core {i}: Shape {core.shape}")

    # Paso 8: Crear el nuevo tensor TT con los núcleos modificados
    tt_tensor_updated = t3f.TensorTrain(new_cores)

    return tt_tensor_updated

def truncate(tensor, target_ranks):
    """Trunca un tensor TT a los rangos objetivo."""
    return t3f.round(tensor, max_tt_rank=target_ranks)

def riemannian_tensor_completion(X_0, A_Omega, Omega, max_iters=10, verbose=False):
    X_k = t3f.orthogonalize_tt_cores(X_0, left_to_right=True)

    target_ranks = t3f.tt_ranks(X_0).numpy()  # Rangos objetivo

    xi_0 = calcular_gradiente_riemanniano_tf(X_k, A_Omega, Omega)
    eta_0 = -xi_0
    alpha_0 = linearized_search(A_Omega, X_k, eta_0, Omega)

    X_temp = X_k + alpha_0 * eta_0
    X_k = t3f.orthogonalize_tt_cores(truncate(X_temp, target_ranks), left_to_right=True)

    xi_k_anterior = xi_0
    eta_k_anterior = eta_0
    ip_xi_xi_old = t3f.frobenius_norm_squared(xi_0)

    for k in range(1, max_iters + 1):
        xi_k = calcular_gradiente_riemanniano_tf(X_k, A_Omega, Omega)
        ip_xi_xi = t3f.frobenius_norm_squared(xi_k)

        eta_transported = t3f.project(eta_k_anterior, X_k)  # Rangos 2r
        beta_k = ip_xi_xi / ip_xi_xi_old if ip_xi_xi_old != 0 else 0

        eta_transported = truncate(eta_transported, target_ranks)

        eta_k = -xi_k + beta_k * eta_transported  # Rangos 4r
        eta_k = truncate(eta_k, target_ranks)  # Truncar a rangos r

        alpha_k = linearized_search(A_Omega, X_k, eta_k, Omega)

        X_temp = X_k + alpha_k * eta_k  # Rangos 2r (r + r)
        X_k = t3f.orthogonalize_tt_cores(truncate(X_temp, target_ranks), left_to_right=True)

        xi_k_anterior = xi_k
        eta_k_anterior = eta_k
        ip_xi_xi_old = ip_xi_xi

    return X_k

# Función para generar nodos de Chebyshev en el intervalo [0, 1]
def chebyshev_nodes(n):
    """
    Generate Chebyshev nodes of the first kind in the interval [0, 1].

    Parameters:
    - n (int): Number of intervals (number of nodes will be n + 1).

    Returns:
    - np.array: Array of (n + 1) Chebyshev nodes in [0, 1].
    """
    k = np.arange(n + 1)  # Indices 0 to n
    q = np.cos(np.pi * k / n)  # Nodos en [-1, 1]
    x = (q + 1) / 2  # Mapeo a [0, 1]
    return x

# Función para generar un conjunto de índices aleatorios (0-based)
def make_omega_set(n_nodes, size, d):
    """
    Generate a set of random multi-indices with d components, each between 0 and n_nodes - 1.

    Parameters:
    - n_nodes (int): Upper bound for each index (range is 0 to n_nodes - 1).
    - size (int): Number of multi-indices to generate.
    - d (int): Number of dimensions.

    Returns:
    - np.array: 2D array of shape (size, d) with random multi-indices.
    """
    if n_nodes < 1:
        raise ValueError("n_nodes must be positive")
    if size < 0:
        raise ValueError("size must be non-negative")

    total_combinations = n_nodes ** d
    if size > total_combinations:
        raise ValueError(f"Requested size ({size}) exceeds total possible combinations ({total_combinations})")

    # Generate random multi-indices (0 to n_nodes - 1)
    Omega = np.random.randint(0, n_nodes, size=(size, d))

    return Omega

# Función para mapear índices a nodos de Chebyshev
def map_to_chebyshev_nodes(Omega, n_nodes):
    """
    Map multi-indices from range [0, n_nodes - 1] to Chebyshev nodes in [0, 1] for each dimension.

    Parameters:
    - Omega: 2D NumPy array of shape (size, d) with multi-indices.
    - n_nodes: Upper bound of the original indices (number of nodes per dimension).

    Returns:
    - 2D NumPy array of shape (size, d) with values mapped to Chebyshev nodes.
    """
    if n_nodes < 1:
        raise ValueError("n_nodes must be positive")

    # Precompute Chebyshev nodes for one dimension (n_nodes points)
    cheb_nodes = chebyshev_nodes(n_nodes - 1)  # n_nodes - 1 intervals give n_nodes points

    # Map each index to the corresponding Chebyshev node
    d = Omega.shape[1]
    Omega_mapped = np.zeros_like(Omega, dtype=float)
    for dim in range(d):
        Omega_mapped[:, dim] = cheb_nodes[Omega[:, dim]]

    return Omega_mapped

def inferir_dimension_entrada(f_vectorized, max_dimension=100):
    """
    Intenta inferir la dimensión de entrada de una función f_vectorized.
    """
    for d in range(1, max_dimension + 1):
        try:
            # Crea una entrada de prueba con la dimensión d
            entrada_prueba = np.zeros((1, d))

            # Intenta ejecutar la función
            f_vectorized(entrada_prueba)

            # Si la función se ejecuta sin errores, la dimensión de entrada es d
            return d
        except ValueError:
            # Si se produce un ValueError, intenta con la siguiente dimensión
            pass
        except IndexError:
            pass
        except Exception:
            # Si se produce cualquier otro error, no se puede determinar la dimensión
            return None

    # Si no se encontró una dimensión válida, devuelve None
    return None

def generate_disjoint_omega_c(Omega, sizeOmega_C, number_nodes):
    """
    Genera un conjunto Omega_C de tamaño sizeOmega_C con índices disjuntos de Omega.
    """
    sizeOmega = len(Omega)
    d = Omega.shape[1]

    if sizeOmega_C > number_nodes ** d - sizeOmega:
        raise ValueError("sizeOmega_C is too large to generate disjoint multi-indices")

    # Convertir Omega a un conjunto para verificaciones rápidas
    Omega = Omega.numpy() if isinstance(Omega, tf.Tensor) else Omega  # Convertir a numpy si es tensor
    Omega_set = set(map(tuple, Omega))

    # Generar más índices de los necesarios para aumentar la probabilidad de encontrar suficientes disjuntos
    Omega_C = []
    candidates_per_batch = int(sizeOmega_C * 1.5)  # Generar 50% más candidatos por lote
    attempts = 0
    max_attempts = 10  # Número máximo de intentos para evitar bucles infinitos

    while len(Omega_C) < sizeOmega_C and attempts < max_attempts:
        # Generar un lote de candidatos
        candidates = np.random.randint(0, number_nodes, size=(candidates_per_batch, d))
        # Filtrar candidatos que no estén en Omega_set ni en Omega_C
        for idx in candidates:
            idx_tuple = tuple(idx)
            if idx_tuple not in Omega_set and idx_tuple not in {tuple(x) for x in Omega_C}:
                Omega_C.append(idx)
                if len(Omega_C) >= sizeOmega_C:
                    break
        attempts += 1

    if len(Omega_C) < sizeOmega_C:
        raise ValueError(f"No se pudieron generar {sizeOmega_C} índices disjuntos después de {max_attempts} intentos")

    return np.array(Omega_C)

# Funciones auxiliares necesarias
def inferir_dimension_entrada(f_vectorized, max_dimension=100):
    for d in range(1, max_dimension + 1):
        try:
            entrada_prueba = np.zeros((1, d))
            f_vectorized(entrada_prueba)
            return d
        except ValueError:
            pass
        except IndexError:
            pass
        except Exception:
            return None
    return None

def adaptive_rank_strategy_Glau_fast(Omega, A_Omega, Omega_C, A_Omega_C, shape, max_rank, rank_initial, verbose, initial_guess=False, X_initial=None, max_iters=10, rho=1e-4):
    '''
    Implementa la estrategia de rango adaptativo rápida basada en el artículo de Steinlechner para la compleción tensorial.
    Permite inicializar el tensor X de dos maneras:
    - Si initial_guess=False (por defecto), X se inicializa como un tensor aleatorio deユTruncado de rango TT [1, rank_initial, ..., rank_initial, 1].
    - Si initial_guess=True, se debe proporcionar un tensor X_initial como entrada, que se usará como punto de partida.

    Args:
        Omega (np.ndarray): Índices de las entradas observadas del tensor.
        A_Omega (np.ndarray): Valores de las entradas observadas del tensor.
        Omega_C (np.ndarray): Índices de las entradas observadas del tensor para validación.
        A_Omega_C (np.ndarray): Valores de las entradas observadas del tensor para validación.
        shape (tuple): Forma del tensor completo.
        max_rank (int): Rango TT máximo permitido.
        rank_initial (int): Rango TT inicial para la inicialización aleatoria.
        verbose (bool): Si True, imprime información detallada durante la ejecución.
        initial_guess (bool, opcional): Si True, se debe proporcionar X_initial. Si False, se inicializa X aleatoriamente. Por defecto es False.
        X_initial (t3f.TensorTrain, opcional): Tensor inicial en formato Tensor Train. Obligatorio si initial_guess=True.
        max_iters (int, opcional): Número máximo de iteraciones (giros de gradiente conjugado) en cada paso de la compleción Riemanniana. Por defecto es 10.
        rho (float, opcional): Umbral para aceptar el aumento de rango. Por defecto es 1e-4.

    Returns:
        tuple: Una tupla que contiene:
            - X (t3f.TensorTrain): El tensor completado en formato Tensor Train.
            - errors (list): Lista de errores en el conjunto de validación (Omega_C) en cada paso de aceptación de aumento de rango.
    '''
    # Convertir entradas a tensores de TensorFlow
    Omega = tf.constant(Omega, dtype=tf.int32)
    Omega_C = tf.constant(Omega_C, dtype=tf.int32)
    A_Omega = tf.constant(A_Omega, dtype=tf.float32)
    A_Omega_C = tf.constant(A_Omega_C, dtype=tf.float32)

    # Número de dimensiones
    d = len(shape)

    # Inicializar X según el valor de initial_guess
    if initial_guess:
        if X_initial is None:
            raise ValueError("Si initial_guess=True, se debe proporcionar X_initial.")
        X = X_initial
    else:
        initial_tt_rank = [1] + [rank_initial] * (d - 1) + [1]
        X = t3f.random_tensor(shape, tt_rank=initial_tt_rank)

    # Optimizar X inicial con Riemannian CG
    X = riemannian_tensor_completion(X, A_Omega, Omega, max_iters=max_iters, verbose=verbose)

    # Calcular el error inicial en Omega_C
    X_Omega_C = t3f.gather_nd(X, Omega_C)
    norm_A_Omega_C = np.linalg.norm(A_Omega_C.numpy())
    error_X = np.linalg.norm(A_Omega_C.numpy() - X_Omega_C.numpy()) / norm_A_Omega_C
    if verbose:
        print(f"Error inicial en Omega_C: {error_X:.6f}")

    # Lista para monitorear el error
    errors = [error_X]

    # Inicializar locked y mu
    locked = 0
    mu = 1

    # Bucle while con condiciones locked < d - 1 y max r_\nu < r_max
    while locked < d - 1 and max(t3f.tt_ranks(X).numpy()[1:d]) < max_rank:
        tt_ranks = t3f.tt_ranks(X).numpy()

        if tt_ranks[mu] < max_rank:
            # Aumentar el rango en la posición mu
            X_new = increase_tt_rank_mu(X, mu, noise_magnitude=1e-8)

            # Optimizar X_new con Riemannian CG
            X_new = riemannian_tensor_completion(X_new, A_Omega, Omega, max_iters=max_iters, verbose=verbose)
            if verbose:
                print(f"Aumento en mu={mu}: Rango TT de X_new: {t3f.tt_ranks(X_new).numpy()}")

            # Calcular el error de X_new en Omega_C
            X_new_Omega_C = t3f.gather_nd(X_new, Omega_C)
            error_X_new = np.linalg.norm(A_Omega_C.numpy() - X_new_Omega_C.numpy()) / norm_A_Omega_C
            if verbose:
                print(f"Aumento en mu={mu}: error_X_new = {error_X_new:.6f}")

            # Criterio de aceptación
            if error_X_new - error_X > -rho:
                locked += 1
                if verbose:
                    print(f"Aumento de rango rechazado para mu={mu}.")
            else:
                X = X_new
                error_X = error_X_new
                errors.append(error_X)
                locked = 0
                if verbose:
                    print(f"Aumento de rango aceptado para mu={mu}.")
        else:
            locked += 1
            if verbose:
                print(f"Rango en mu={mu} ya es max_rank.")

        # Actualizar mu cíclicamente
        mu = (mu % (d - 1)) + 1

    if verbose:
        print(f"Error final en Omega_C: {error_X:.6f}")
    return X, errors

def adaptive_sampling_strategy_1(f_vectorized, sizeOmega, sizeOmega_C, number_nodes, rank_initial, max_rank, p=0.2, tol_1=1e-4, tol_2=1e-8, max_iters=10, rho=1e-4, verbose=False):
    d = inferir_dimension_entrada(f_vectorized)
    n = number_nodes * np.ones(d, dtype=int)
    shape = tuple(n)

    # Paso 1: Crear conjunto de prueba inicial Omega_C_new disjunto de Omega
    Omega = make_omega_set(number_nodes, sizeOmega, d)
    Omega_mapped = map_to_chebyshev_nodes(Omega, number_nodes)
    Omega_C_new = generate_disjoint_omega_c(Omega, sizeOmega_C, number_nodes)
    Omega_C_new_mapped = map_to_chebyshev_nodes(Omega_C_new, number_nodes)

    # Generar valores evaluando la función en los puntos mapeados
    A_Omega = f_vectorized(Omega_mapped)
    A_Omega_C_new = f_vectorized(Omega_C_new_mapped)

    # Convertir a tensores de TensorFlow
    Omega = tf.constant(Omega, dtype=tf.int32)
    Omega_C_new = tf.constant(Omega_C_new, dtype=tf.int32)
    A_Omega = tf.constant(A_Omega, dtype=tf.float32)
    A_Omega_C_new = tf.constant(A_Omega_C_new, dtype=tf.float32)

    # Paso 2: Ejecutar Algorithm 1 para obtener X_c inicial con inicialización aleatoria
    X_c, _ = adaptive_rank_strategy_Glau_fast(
        Omega, A_Omega, Omega_C_new, A_Omega_C_new, shape, max_rank, rank_initial,
        verbose=verbose, initial_guess=False, max_iters=max_iters, rho=rho
    )

    # Paso 3: Calcular error inicial en Omega_C_new
    X_Omega_C_new = t3f.gather_nd(X_c, Omega_C_new)
    err_new = np.linalg.norm(A_Omega_C_new.numpy() - X_Omega_C_new.numpy()) / np.linalg.norm(A_Omega_C_new.numpy())
    if verbose:
        print(f"Iteración 0: |Omega| = {len(Omega)}, Error en Omega_C = {err_new:.6f}")

    # Paso 4: Bucle mientras |Omega| / size(A) < p
    iteration = 0
    while len(Omega) / np.prod(shape) < p:
        iteration += 1
        # Paso 5: Guardar error anterior
        err_old = err_new

        # Paso 6: Aproximar X_c a rango (1, ..., 1)
        X_hat = t3f.round(X_c, max_tt_rank=rank_initial)

        # Paso 7: Guardar Omega_C_old
        Omega_C_old = Omega_C_new
        A_Omega_C_old = A_Omega_C_new

        # Paso 8: Crear nuevo Omega_C_new disjunto de Omega_C_old
        Omega_C_new = generate_disjoint_omega_c(Omega_C_old, sizeOmega_C, number_nodes)
        Omega_C_new_mapped = map_to_chebyshev_nodes(Omega_C_new, number_nodes)
        A_Omega_C_new = f_vectorized(Omega_C_new_mapped)

        # Convertir nuevos conjuntos a tensores
        Omega_C_new = tf.constant(Omega_C_new, dtype=tf.int32)
        A_Omega_C_new = tf.constant(A_Omega_C_new, dtype=tf.float32)

        # Paso 9: Actualizar Omega con Omega_C_old
        Omega = tf.concat([Omega, Omega_C_old], axis=0)
        A_Omega = tf.concat([A_Omega, A_Omega_C_old], axis=0)

        # Añadir print aquí para mostrar el tamaño de Omega
        print(f"Iteración {iteration}: Tamaño de Omega = {tf.shape(Omega)[0].numpy()}")

        # Paso 10: Ejecutar Algorithm 1 con X_hat como punto inicial
        X_c, _ = adaptive_rank_strategy_Glau_fast(
            Omega, A_Omega, Omega_C_new, A_Omega_C_new, shape, max_rank, rank_initial,
            verbose=verbose, initial_guess=True, X_initial=X_hat, max_iters=max_iters, rho=rho
        )

        # Paso 11: Calcular nuevo error en Omega_C_new
        X_Omega_C_new = t3f.gather_nd(X_c, Omega_C_new)
        err_new = np.linalg.norm(A_Omega_C_new.numpy() - X_Omega_C_new.numpy()) / np.linalg.norm(A_Omega_C_new.numpy())

        # Paso 12: Verificar criterios de parada
        tt_ranks = t3f.tt_ranks(X_c)
        rank_max_reached = any(r >= max_rank for r in tt_ranks[1:-1])

        if verbose:
            print(f"Iteración {iteration}: |Omega| = {len(Omega)}, Error en Omega_C = {err_new:.6f}, Rango TT = {tt_ranks.numpy()}")

        if (err_new < tol_1 or
            abs(err_new - err_old) < tol_2 or
            rank_max_reached):
            break

    # Paso 16: Asignar tensor final
    X = X_c

    return X

---

# Ideas for improvement

Maybe implement a very basic tensor completion for the initial guess of the algorithm, instead of starting with a random tensor.